# Angular Concepts Brain Dump

These are things I remember from working with Angular at Google that aren't always apparent in tutorials or snippets that I didn't want to forget over time. This is just a brain dump and should be explicitly fact checked to make sure I didn't remember anything wrong.

- A component instance variable must be **public** for the template to be able to reference it in **interpolation**.
  - also, in the template, you **don't use `this`** for the component instance variables
- If I remember correctly, a **directive** is basically a component without a template (behavior only). They are normally attached via attributes, but technically components could be as well (though that's less common).
- The **selector** for a component is usually taken as meaning the element name that will become that component, but technically it works this way:
  1. When the template is compiled, all tags and attributes are checked against either the HTML spec or the selectors from component and directive metadata.
  2. Any that don't match one of those two things cause a compile error.
  3. Selectors for components are used to determine which components and directives will be created on which elements.
     - I don't remember if this happens at compile time or runtime. Some parts of components are static but some are dynamic, so maybe a little bit of both?
  4. The tag that a directive or component is attached to is the **host** element and remains in the DOM. Anything added by the directive or component, such as classes added in the metadata in TypeScript, or a template added by a component, is added to the DOM under the host element.
- **Style encapsulation** means that styles applied to one component don't leak into others. If I remember correctly, in a normal debug build, this is accomplished by adding special classes to all the style selectors and to the host element. In a minified/optimized build, it uses the shadow DOM (the details of which I have forgotten). It is configurable which kind to use when.
  - `::ng-deep` is a keyword you can put in front of a selector in your style for a component to turn off encapsulation for that selector. It is usually not recommended, but there are some cases where it can't be avoided.
    - eg. You generate a popup window that you need to style. Since it isn't part of the DOM tree of the component, you can't use style encapsulation.
  - `::ng-host` is another useful css keyword for applying stylings to the component's element itself instead of having to search children
    - eg. so that you can do `display: block` for the top-level element in a component
  - you can apply styles in the **global styles** for the app and then no encapsulation will be applied to them. If you want to be more selective about where global styles get applied, you can add your own classes inside templates that are aimed at the global styles.
- **Rendering** of an Angular app is done via JS code generated by the compiler. Even though component templates look like HTML, they get compiled into JS (which actually builds the page more efficiently than pure HTML). The JS executes when the page is loaded and creates the DOM, which looks similar to the templates but with the dynamic stuff replaced by real stuff.
- **Boostrapping** is the process of running the Angular app in the page. The JS code generated by the angular compiler needs to be included via a \<script\> tag on an html page in order to run the app in a browser. In the past I saw this done by creating an index.html that wasn't part of the build, and including a script tag to bring in the output JS. Looking at the newer generated skeleton projects, this seems to be done slightly differently now.
  - I would imagine it's still pretty flexible because you may want to serve your JS code via a CDN, etc.
- **Unit tests** are in Jasmine syntax (describe, it, spyOn), run by Karma. There's also Protractor, which I do not have experience with. Screendiff and integration testing (eg. CUJs) can be done via **Selenium**. I have experience with that but in a way that is not applicable to the real world, so I still need to learn the real world way.
  - **matchers** are functions that return something that jasmine can use to decide if an object returned by a test maches a condition. You can use the built-in matchers like toEqual(), or import other matchers from libraries, or make them yourselves. Somehow, it ends up so that you can use matchers as functions in-place in some calls (like .toBeCalledWith(any(Object)), or use them in expect() statements `expect(bla).toBe(5)` (uses toBe matcher). I don't remember the exact mechanism here - presumably matchers are somehow decorated or declared somewhere that jasmine looks for and added to the prototype of some object returned by expect()?
- **Dependency injection** lets you get things like **services** (which are just classes that you mark as injectable and can assume are singletons) as members of component classes by adding them as constructor args. Angular constructs components instead of you, and it knows how to inject based on its configuration. You would usually configure a service to be injectable forRoot, but it's possible to do more complicated configuration such as matching tokens (eg. if need to inject multiple variations of the same type of thing). This is done via modules.
- **Modules** are the way Angular is configured to understand new tags (via components included in module), pipes, injectable objects, etc. Modules import other modules, declare components, export components, etc. **CommonModule** is the one that adds basic Angular stuff like (click), \*ngFor, etc. although for some reason that is not present in the default app and it still works.
- [] is for **dynamic binding** in general (attribute set at runtime based on TS code in quotes)
  - it can be used for any attribute (I think)
    - even attributes that are used as selectors for **directives** are ok to set dynamically with []
  - it can be used to set a class with [class.classname]
  - it can be used to set an attribute with [attr.attributename]
  - it can be used to set an input on a component (the most common way you'll see it
  - if you just need to pass a literal string, don't use [], because that will force you to nest quotes for no reason
- () is for **event binding** in general
  - it is used to execute the TS code in quotes when an event triggers
  - generally the event is either built into Angular (like 'click' on buttons) or is an `@Output()` of a component
  - from the perspective of the code that executes on an event, the `$event` value gets set in the TS code that runs and you can pass that to a TS method call if needed
  - when you define your own event via @Output(), you are defining an observable, and the emitted values become $event one by one on anyone using () to bind to them
- `[(ngModel)]` ("banana in a box") is for **two-way-binding** where updating the form field will update the variable bound to it and vice versa (without infinite recursion happening)
  - If I recall correctly (though I'm mostly thinking of Dart I think), this kind of syntax in general means you're binnding to an input and output that happen to have the same name at the same time, and they each are implemented in the directive/component to receive inputs and send outputs correctly while updating internal state
  - although contrary to that, I've seen somewhere else that it literally takes what you put in quotes and adds = to it, so maybe I'm wrong
- template TS syntax (eg. in {{ }} for interpolation or in a [] or () expression) is a subset of TS
  - generally can be an expression that returns a value, when applicable
  - doesn't need to end with ;
  - can use ; to separate multiple statements though
- the **JS event loop** can't continue on to the next thing until your TS code completely returns from whatever it was handling
  - so if you change an input to a component and then read something else on the component before returning from a click handler, change detection has had no time to run yet
  - You can inject `ChangeDetectorRef` and run `detectChanges()` if you want to do an inline change detection change without returning back to the queue
  - You can also use `markForCheck()` to make sure it runs change detection on something you changed (in a way that Angular won't see), but this will run on the next ordinary change detection run (eg. from the event queue)
- **Schematics** are a way to customize templated generation of files, such as components in your team's style, etc.
  - I don't remember the low-level details, but you configure somewhere that when you run a command like `ng generate [something]`, the `[something]` part is a schematic (I think)
- **source maps** are for allowing chrome devtools or another debugger to link the browser's JS code with your TS code
  - similar in concept to C++ symbols


# npm Packages to Know

- **typescript**
  - the TypeScript language compiler, etc.
- **ts-node**
  - interpretter for TypeScript
- **prettier**
  - formatting for TypeScript and a lot of other languages
- **eslint**
  - linter for JS that can be configured to lint TS
- **@angular/cli**
  - Angular CLI plus the main Angular libraries, including forms, routing, etc. etc.
- **angular-in-memory-web-api**
  - lets you mock an HTTP web server so you can write real HTTP code in your app but use hard-coded data localy
- **json-server**
  - lets you mock an HTTP web server using a json file instead of in-memory
- **uuid**
  - lets you generate unique IDs for things like form controls so that they don't clash with other components
- **@ngrx/\***
  - ngrx state management, spread across multiple packages, with `ngrx/store` being the main one
  - in-memory app state (goes away on browser refresh)
    - store holding the state, actions dispatched to store, reducers changing the store based on actions, effects doing side effects based on actions

# Angular CLI

NOTE: remember in all these examples, **don't put the word** directive, component, module, etc. in the name of the thing because that will be added already. It was done here just to make it clear what each token was meant to be for in the CLI examples.

`ng new my-app`

- creates a folder called _my-app_ in the current folder and initializes it as a default Angular app with a component, module, config files, etc.

`ng new my-app --routing`

- automatically says yes to the questiona bout routing
- gives you a separate routing module imported by the main module, with empty route paths to fill in

`ng new my-workspace --create-application=false`

- creates a folder called _my-workspace_ with top-level Angular files, but doesn't put an app in it yet.
- has some things already like angular.json and tsconfig.json

`ng generate component my-stuff`

- creates a folder called my-stuff in the app and puts component files in it (with selector app-my-stuff which you can easily change) using class name MyStuffComponent, wiring it into the main app module

`ng g c my-stuff` - shorter way to do `ng generate component`

- try other abbreviations for fun

`ng generate component subfolder/my-stuff`

- put component under specific subfolder (and then its own my-stuff folder)

`ng generate component subfolder/my-stuff --flat`

- the `--flat` option means don't make a subfolder for the component itself (but in this case will still go into subfolder)

`ng generate component my-stuff --module=some-module`

- changing the module that the generated component goes into

`ng generate service my-service`

- generating an injectable service class and test
- by default not in a subfolder, unlike components (flat already)
- all the above options (subfolder, etc.) apply

`ng generate directive my-directive`

- generating a directive and test file
- by default not in a subfolder, unlike components (flat already)
  - the reason is because directives are in 1 file instead of multiple
- all the above options (subfolder, etc.) apply

`ng generate module my-module`

- generating a module (eg. a separate module for routing)
- can specify module (eg. your app module) to import it into
- same options as above in terms of subfolders, flattening, etc.

`ng generate interface my-interface`

- generates an interface file in the root of the app src
- not really that much easier than just making a file and making an interface, but it's available

`ng generate guard my-guard`

- creates a route guard (asks you which type)
- then you have to manually add it to the routing module where it's needed
- just a TS file and a test file, similar to service (but not injectable)

`ng generate resolve my-resolve`

- similar to `ng generate guard` but for a `resolve` guard

`ng generate application my-app`

- generates an application called _my-app_ in the current workspace (as created by `ng new`)
- workspace may or may not already contain other apps and libraries
- by default, goes into _projects_ subfolder (configurable in _angular.json_)

`ng generate library my-lib`

- generates a library called _my-lib_ in the current workspace (as created by `ng new`)
- workspace may or may not already contain apps and libraries
- by default, goes into _projects_ subfolder (configurable in _angular.json_)

`ng generate component --project=my-lib my-component`

- if have more than 1 project, use _project_ flag to specify which to add a new thing to

`ng add @angular/material`

- uses the `add` schematic to download a package and install it into the app
- this is beyond what just an npm install would do
- in this example, it is the Angular Material package, which will give you theming, Angular Material components, Angular Material CDK, animations, etc.
- it is also possible to add a subset (eg. `ng add @angular/cdk`)
- once installed, this will add several **other schematics** (eg. `ng generate @angular/material:navigation my-nav`) to get pre-generated components from templates with mock data

`ng serve`

- compile and serve the app from the current working directory
- app will be on a localhost port and the url will be posted to the console
- will (usually) automatically rebuild and refresh when changes are detected
- on Windows WSL, that sometimes doesn't work for me and I have to use --poll
- it seems like certain global changes like changing the global styles doesn't auto-refresh - maybe only the compiled JS triggers a refresh

`ng serve --poll 1000`

- if auto-compile and refresh is not working after files are changed, try this
- will cause it to check the files itself every **1 second**

`ng serve --open`

- automatically open browser pointing to the served app
- even works on WSL w/ Windows version of Chrome

`ng test`

- run tests for project of current working directory
- by default karma runner for jasmine tests
- like _ng serve_, it should keep refreshing when you make changes
  - thus you can keep the tests running all the time and change things and see if anything breaks
- like _ng serve_, I have had issues in WSL and have to use --poll

`ng test --poll 1000`

- similar to _ng serve_ above, has it look for file changes every **1 second** in case automatic file system watching is broken

`ng test --browsers=ChromeHeadless`

- run tests without having to open browser (faster)
- all output visible in terminal
- requires extensive setup of system before you can use it

`export CHROME_BIN=/mnt/c/Program\ Files\ \(x86\)/Google/Chrome/Application/chrome.exe`

- if `ng test` has trouble finding the Windows chrome on WSL in Windows, try this
- for me, it had trouble sometimes and not others, so I added this to my bashrc

`ng serve --configuration=production`

- `ng serve` and `ng test` both have an option to specify a non-default configuration, configured in _angular.json_.

`ng build`

- builds into `dist` folder

`ng build --configuration=production`

- the official way to build a production-ready app to serve
- it is not recommened to use the one built by `ng serve` in production
- configuration comes from your `angular.json` file
  - usually you start with a production and development config
- NOTE: projects generated by Angular CLI nowadays actually **default to production** and you'd have to override to get a development build
  - this makes sense because `ng serve` builds in-memory instead and doesn't use this build, so you'd mostly only use it for production

`ng build -c production`

- same as --configuration=production

`ng build my-lib`

- building a specific project within the current workspace
- note that it's _by name_ instead of path

`ng test my-lib`

- testing a specific project by name

`ng serve my-app`

- serving a specific app by name


# References

[angular.io](http://angular.io)

[material.angular.io](http://material.angular.io)

[colors](https://m2.material.io/design/color/the-color-system.html#tools-for-picking-colors)


# Some Important Concepts

These are not well-illustrated by looking at tutorial snippets but are important to know.


## Files

- `package.json`
  - NPM package management for the angular project/workspace
  - also custom npm scripts to run from the shell
- `package-lock.json`
  - goes along with `package.json`
- `angular.json`
  - configuration related to the angular project/workspace such as locations of projects and how/where artifacts get built
  - also things like debug vs. prod config
- `tsconfig.json`
  - for TypeScript compilation options (lower level than angular.json)
- `tsconfig.app.json`
  - extensions to tsconfig.json for app
- `tsconfig.spec.json`
  - extensions to tsconfig.json for tests

## Template References Variables

### Normal Usage

```html
<div #myvar>
  <button (click)="dostuff(myvar)">click</button>
</div>
```

In this case, you can use that name `myvar` elsewhere in the template to refer to the HTML element it is on. In this example, it was passed into a method on the component which will be expecting an `HTMLElement` to do some stuff with.

### On Component

```html
<my-component #myvar>
  <button (click)="dostuff(myvar)">click</button>
</my-component>
```

In this case, because the element is a component, the component is passed into dostuff() instead of the HTML element, allowing you to **access its members**.

### With Argument

```html
<form #myform="ngForm">
  <div *ngIf="myform.valid">...</div>
</form>
```

In this case, the reference variable is bound to an **exported directive name** that has been applied on the element. When you include **FormsModule** in your app module, all `<form>` tags are marked with the **NgForm Directive**, which exports `ngForm` for this usage.

### Variable Scope

In general, the variable is scoped to the whole template (even things before it was declared). But certain structural things like `*ngIf` and `*ngFor` create a deeper lexical scope that hides it from above.


## Forms


### Template-Driven Forms

#### ngForm

When you import `FormsModule`, any `<form>` tag in your template will be marked with an `NgForm` directive, which can be accessed from a template reference variable as shown above, to access things like overall form validation state, etc.

You can also listen to the `(ngSubmit)` event on a form tag to trigger when a form is submitted. I've also seen people just listen to the `(click)` of the button itself without problems (as long as you update whatever state you are caring about).

Ngform looks for all form controls under it and uses their `name` attributes to build an internal representation of the form state. It also understands that button of type _submit_ will submit the form (trigger the ngSubmit event). Note that HTML `<label>` attributes look at `id` instead of `name`.

#### ngModel

In addition to being for two-way binding, HTML form control elements within a form wrapped by NgForm also get wrapped as NgModel instances. You can access the state of individual form controls with `#mycontrol="ngModel"`.

Note that using two-way binding does not depend on being inside a form tag.

#### Validation

Validation is happening all the time (after a user enters some input for instance). It is not directly tied to submission. For instance, if a field is empty and you have a `required` tag (from HTML spec, not from Angular) on the field, the control's `valid` property will be false, and so will the overall form.

There are two ways you can react to the validity:

1. via the `valid` properties of NgModel and NgForm.
2. via `ng-*` css classes (see below).

The built-in HTML form validation is disabled via `novalidate` on the form directive. This can be configured if needed. Angular replaces the built-in validation.

#### Submission

A button that is marked as type `submit` will submit the form. All that means is that the state is changed to `submitted`. You have to listen for the event and do something useful with that.

#### CSS Classes

NgForm adds css classes to the `<form>` and to child controls so that you can style them based on the validation state and such.

Examples are:

- ng-touched
- ng-dirty
- ng-pristine
- ng-valid
- ng-submitted
- etc.

Both positive and negative versions are provided to make CSS styling easier.

#### Resetting

`form.reset()` on Ngform will reset the state and values of the form. If the form was guraded by `*ngIf` and has two-way binding, this will probably just reset the state and the values will be correct. If the form was guarded by `[hidden]` instead, resetting will probably reset the values too, because change detection won't run if the form's inputs were not changed.

#### ID Uniqueness

If an app is large and has the possibility of form ID clashes, you may want to generate unique IDs instead of hardcoding them. You can `npm install uuid` and then import `uuid.v4()` into the component TS file. Then call it to initialize a field for the ID of each form control.

In the template, you can dynamically bind the `[id]` of the form control and the `[for]` of the label to this unique ID.


### Reactive Forms

#### In Contrast to Template-Driven Forms

Template-driven forms take the html template as the source of truth and communicate with TS via two-way binding.

Reactive forms have a model (officially called the **form model**) inside the TS of a component as the source of truth, and the template is configured to bind to that. Reactive forms deal more with **observables**, **interfaces**, etc. It is technically possible (but confusing and a bad idea) to mix the two approaches since they use a common infrastructure.

#### Advantages of Reactive Forms

- immutability of objects (as opposed to two-way binding) = cleaner
- more efficient change detection (due to the above)
  - and don't have to trigger change detection as many times in tests because of how it works
- more scalable and dynamic

#### Disadvantages of Reactive Forms

- not as straightforward for simple cases (eg. a handful of control doing a simple input form)

#### Parts that are the Same

Reactive forms and template-driven forms are the same in terms of:

- **CSS classes** like ng-valid
  - CAVEAT: if you style using the 'required' attribute, but you specify the attribute in the model, it won't work
    - to fix that, add a 'required' class in the template or something like that
- **testing** is mostly the same (finding controls in the DOM and sending events, etc.)
  - but you may not need to call fixture.detectchanges() and/or fixture.whenStable() so many times
- **submitting** works the same (button with 'submit' style and ngSubmit event)

#### Resetting Form

A nullable field will reset to `null` instead of the initial value on `formGroup.reset()`. This can be counterintuitive. You can also make it **non-nullable** to prevent that from happening (shown in the commits below).

#### Basic Idea

You create `FormControl` instances in code, and group them with `FormGroup` and `FormArray` as needed to create a model of the form. Rather than binding html form controls to objects in the template, you bind to names. The names match keys of model objects in code. You just need 1 top-level form group bound to the whole model in code, and then the child elements will find their place in the model tree.

You can use methods of the model objects like `setValue()` and subscribe to the value changes as an observable, catching changes that happen either in code or in the UI. All this is shown in the commits below.


### ControlValueAccessor

If you have a **custom component** that you want to act like a **form control** (eg. the way the built-in html form elements do), you can implement the **ControlValueAccessor interface**. This interface tells Angular how to get/set values for the component in the right format and stuff like that.

After implementing the interface, then you can use the component in a template and use a directive like `formControlName` to bind it to a form control in the model (for reactive forms). You could use it similarly to an element like input (ngModel, etc.) in a template-driven form.


### Special Kinds of Custom Validators

These are not shown in the snippets below, but it's good to know they exist.

- **cross-field** - works same way as a normal validator but is specified in a form group instead of a control
  - then the validator knows which controls it wants to read from the group passed in
- **async** - for things like going to the server to check if a name is already in use
  - deals with observables and `model.pending` so you can show a spinner or something
  - https://angular.io/guide/form-validation#creating-asynchronous-validators


## ng-container

This is just a special element you can add in your template that **doesn't add** anything to the **DOM**. This can be useful if you don't want to accidentally mess up alignments or styling by adding an extra div, for instance.

The purpose of the element would be to hold things like `ngIf` to block child elements from existing.


## async pipe

The async pipe automatically handles subscribing to observables, chaining promises, etc. so that you can treat them as a single synchronous value in a template.

You will probably need to do an extra `fixture.detectChanges` in your test to interact with a field where an async pipe is used.

## Change Detection

### Overall Process

1. When a triggering event occurs (user interaction event, observable emitting, http request completing, setTimeout lambda being called, etc.), run change detection process at top of tree (eg. parent component).
   - note that change detection runs **after the handlers** for events, so that you can change bindings in event handlers and have them reflect right away
2. Check parent component for input binding changes or field changes. The exact checking depends on which strategy the component is configured to use.
3. If any relevant change is found, mark the component dirty and check its children using their strategies, and so on.
4. Call ngOnChanges for dirty components.
5. Re-render dirty components (presumably from the bottom up).

### Strategy

- **Default** strategy will check all input bindings and member variables in a value-comparing way
  - if a field in an object (even if nested) or an array value is changed, it will see it
  - this is the most expensive but allows for full mutability of data
- **OnPush** strategy will only check for certain kinds of changes to input bindings and member variables
  - essentially it will do an ===
    - which means primitives will be value compared, but objects and arrays will be referenced compared
  - much less expensive
  - encourages immutability which is cleaner

### Triggering Default-style Change Detection With OnPush

There are two ways:

1. Change the object reference of an input binding of the component to be triggered by using the **slice operator** to copy the object (eg. you could use a **getter** to make this transparent and use immuatable style).
2. Inject `ChangeDetectorRef` and call `markForCheck()`.

### ngOnChanges

In theory, ngOnChanges is supposed to hook all change detection dirtiness, but in my testing, it only detects what OnPush change detection would pick up, even if you're using default. Thus, you can see a lot of re-renders that are not signaleld by ngOnChanges.

ngOnChanges runs in between checking for dirtiness and updating the view.

### Parent-Child Propagation

Change detection normally flows down from the root component and only checks children whose parents have been marked dirty. However, it is still possible for children to be dirty when their parents aren't, such as when using `markForCheck()`.

If a parent has OnPush, that may restrict when the child gets to run based on changes to the parent.

### ChangeDetectorRef

You can inject ChangeDetectorRef in your component and call methods to manage change detection for the component.

`markForCheck()` will cause a component to be checked for changes using the **Default** strategy (even if it's an OnPush component) regardless of the parent status. It can be used for the following scenarios:

- if the code has to make a change to a property of an object and OnPush is being used
  - an alternative to doing this would be to use slicing to copy the object so that its reference is different
- if some async code such as an http call or observable needs to change the state of the object independently of any bindings from the parent

`detectChanges()` manually triggers the whole change detection process on a component and its children. It is used in conjunction with `detatch()` to take full control over when change detection occurs, such as when change detection is very expensive for a component and you want to make it happen at certain intervals.

### Implementation Details

In JS (and thus TS), you can patch any symbol with your own version and call the original as a wrapper. There is a library called **Zone.js** that does this for Angular. It wraps all the major event handlers so that the originals still happen but with Angular also doing its own stuff such as change detection.

A side effect of this is that APIs that aren't known to Zone.js will not trigger change detection.

Note that because of the way it's designed, there are **no cycles**. A binding change may cause a view change, but it won't feedback and cause more binding changes. Two-way binding in forms is designed to not cause a cycle.

Note that there is an **injectable service** called **NgZone** that can be used to run code that is not in the Angular zone inside the Angular zone.

`this.ngZone.run(() => { this.button.click(); });`

This is usuaully used for handlers for 3rd party asynchronous libraries. You may have to run `changeDetectorRef.detectChanges()` within the same handler depending on the situation.


## Routing

When your server receives a request on a URL, the **server-side** needs to know that paths under your app should all get served with the angular app.

Then once the client code loads the Angular app in the browser, the portion of the URL that looks like a subpath of the app can be read via JavaScript by Angular internally. This information is not used until it is needed - otherwise the app and components load as normal.

When there is a `router-outlet` tag in one of your templates, then the routing information gets used. The url is matched against routing paths in order until one matches, and then a component is loaded after the router-outlet.

If the user **manually** changes the url in the address bar, it's like starting over (whole app **reloaded**, but with the correct routes).

If the url is changed from within the app (eg. via a button w/ `routerLink` or via `router.navigate()`), the component is switched out without reloading the app. If the component doesn't need to be changed but an input does (eg. id in the url), the component can listen to an observable to react to the changes.

NOTE: if you ever need a custom url matcher, check this example out: https://angular.io/guide/routing-with-urlmatcher.

Generally, routing is done in **separate modules** and imported into app modules. An example is to have a module for the main app routing and then other modules imported by that module for individual features within the app. Routing modules tend to be parallel to feature modules. Routing can be relative to features (see the tutorial commits below).

You can do things like only allow users to navigate to routes for which they are authorized via **route guards**.


## NativeElement vs. DebugElement vs. ComponentInstance


You mostly see these come up in tests, but they also come up in component code (eg. when you use `ElementRef`).

The test fixture can give you all of these things for the component under test, plus a lot more.

`ComponentInstance` is the component class instantiated and attached to the DOM element.

`NativeElement` is the JS DOM element that you can interact with the old-fashioned way. You can find its children with `nativeElement.querySelector()` and `nativeElement.querySelectorAll()`, using css selectors.

`DebugElement` provides enhancements beyond native element, such as being able to query by directive in addition to css selectors.

Here is an example of the syntax: `const element = debugElement.query(By.directive(MyComponent));`


## Testing Minified Code

Configurations such as minification, optimization, etc. are done via configurations in _angular.json_.

By default, `ng serve` has a _development_ and _production_ configuration, defaulting to _development_. You can run production with `ng serve --configuration=production`.

By default, `ng test` only has 1 configuration, so you'd have to make another and then run it with **--configuration** like above.


## Libraries

See [here](https://angular.io/guide/creating-libraries) for how to make shared libraries of components and directives. Basically, you make a workspace via `ng new` and then add one or more libraries to it with `ng generate library` and then to use that in an application, you have to `ng build` and `ng pack` it and then import as an `npm` package via a registry.


## Workspace

The default way of creating an app as shown in tutorials creates a workspace that is tied to an app. It is also possible to create a workspace that can contain multiple apps, shared libraries, etc. in one workspace. This can streamline things like using a shared library without having to package first.

You can also treat an existing app as a workspace and use `ng new application` inside to create more apps.

When in the workspace in the shell, you would **specify the app name** to commands like `ng serve`.


## Angular Material

Certain global appearances are set globally via **global styles** just by inclusion of Angular Material in the app via the schematic. Other things, such as mat-button appearance, are set via **attributes** (and via importing modules in the component using them).

Each theme has a primary, accent, and warn **palette**. There are numerous functions and variables defined in **scss** for getting colors from palettes/themes. You can see [here](material.angular.io/guide/theming) for more details.

The **CDK** has behavior for implementing your own components, and the **generate schematics** have several pre-built components (with mock data to help you get started) that you can add to your app.

To test an app that has Angular Material components, you can use the **test harnesses** provided by Angular Material.

```js
import {HarnessLoader} from '@angular/cdk/testing';
import {TestbedHarnessEnvironment} from '@angular/cdk/testing/testbed';

let loader: HarnessLoader;

describe('my-component', () => {
  beforeEach(async () => {
    await TestBed.configureTestingModule({imports: [MyModule], declarations: [UserProfile]})
        .compileComponents();
    fixture = TestBed.createComponent(UserProfile);
    loader = TestbedHarnessEnvironment.loader(fixture);
  });
}
...
import {MatButtonHarness} from '@angular/material/button/testing';

...

it('should work', async () => {
  const buttons = await loader.getAllHarnesses(MatButtonHarness); // length: 3
  const firstButton = await loader.getHarness(MatButtonHarness); // === buttons[0]
});
```


## Setter as @Input

You can have a setter for an @Input member and do something like publish each new value for the input to an observable.


## Projects/Libraries

_angular.json_ is present at the root of an Angular workspace and has a a _projects_ field that lists the projects in the workspace, including apps and libraries. When you create an application with `ng new` or add a project with `ng create`, the Angular CLI modifies this file in addition to adding the files. The type of each project is specified in its entry in _angular.json_, as well as other settings.

Each project has _architect_ settings that specify what to do when _ng build_, _ng serve_, and _ng test_ are executed, as well as configurations that can be passed in via _--configuration_. The current way it works (don't be confused if you see old articles saying it worked a different way) is that ng commands trigger architect entires in a workspace for a project via angular.json, and those entries tell it what _builder_ to use (generally the ones installed with Angular). This architecture allows for you to call the builders on your own as well while still using the same configuration - this is what makes Bazel integration possible without having to sacrifice use of the CLI. _angular.json_ also specifies a _default configuration_ for each project (defaults to _production_).

Note that `serve` does not use `build` - serve builds the artifacts in-memory and defaults to _development_ while build builds the artifacts on disk and defaults to _production_.

By default, new projects go into a _projects_ folder, but this is configured via _newProjectRoot_ in _angular.json_.

A library is just a project that has no entry point and can't be run/served. It can be built and tested.

The Angular CLI refers to projects (both apps and libs) by their top-level names in commands like `ng build` and in scaffolding like `ng generate component --project=my-app my-component`.

To import a library into another library or an app, you just import from it by top-level name in the import statement (no relative path or anything like that needed).

To put projects into different subfolders (eg. to move apps into _apps_ and libs into _libs_), you have to look for hardcoded/relative paths in the following places:

- angular.json
- tsconfig.json
- BUILD files
- custom .bzl files
- karma.conf.js

Angular CLI does not automatically build/rebuild libraries when you build/serve libraries/applications that use them. You need to explicitly `ng build` each dependency to see the changes. It is as if each project has an independent release schedule. When an app is running via `ng serve`, only changes to the app code itself will trigger refresh (not the libs it depends on). It is possible to mitigate this by using _Bazel_.

Each library has a _public-api.ts_ file that specifies what will be exported from the library.

_NPM_ dependencies are installed for the _whole workspace_ instead of individual projects. Individual projects can import any dependency installed in the workspace. Presumably, a minified build would tree-shake the unneeded ones out.


# Examples


## Tour of Heroes

I recently did the [Tour of Heroes Tutorial](https://angular.io/tutorial/tour-of-heroes) and carefully commited one small piece at a time for the following purposes:

- to have a reference for myself and others of small concepts over time
- to have all concepts discussed in the tutorial represented, even ones that are taken out by the final product (eg. @Input() doesn't survive to the end but I have a commit showing it still)
- to do unit tests in the same commits to show how each concept can be tested as well as implemented
  - this is not part of the tutorial - I had to go from memory and look up a lot of things that I either forgot or that work differently on the outside
- to get acquainted with how all this works outside of Google

The GitHub copy is [here](https://github.com/davidpet/tutorials/tree/master/Angular/tour-of-heroes-extended).


### Differences in My Version

- all changes are unit tested, even transient ones that will go away (because the commits are standalone snippets to me)
- I use the `takeUntil\(this.destroy$\)` pattern to clean up observable subscriptions
- I fixed a bug in the hero search service where they were trimming part of the time but not consistently


### Implicitly Done in All Commits

These things were done in all commits but not mentioned below because they're just assumed:

- new & modified tests -> all tests pass before commit
- formatting w/ prettier
- CSS styling directly from them (I didn't change any)
  - there's a bug in my final product where the title text is the wrong color
  - I don't know why that is but I'm using their exact styling, so I'm guessing they fixed it in their own live server but never checked it in
    - It's probably something simple like a slightly wrong selector -> I'll get to it sometime


### 0. Creating App


#### Default App


`ng new app-name` creates initial skeleton with `AppComponent`, `AppModule`, etc.

It asks you questions about what kind of style sheets, routing, etc. you want.

[GitHub: Clean 'ng new' application w/ CSS and no routing.](https://github.com/davidpet/tutorials/commit/5c35e8949c9a128a964af04cbc5847dcaf4acd76)


#### Showing Variable (Interpolation)


[GitHub: Step 0: Create a Project](https://github.com/davidpet/tutorials/commit/19cb4f13f5f7b273d0ceaa74b6761c71221cd998)


### 1. Hero Editor


#### Generating Component


`ng generate component heroes` creates `HeroesComponent` under its own subfolder (along w/ styles, template, and tests) and adds to the main application module.


[Step 1A: Adding heroes component](https://github.com/davidpet/tutorials/commit/93a6c27d79589e77bc9ea6c653af266812e181c7) - add component, include it in main page


#### Two-Way Binding


Binding a component variable with `[(ngModel)]` (banana in a box)

[Step 1B: Adding hero interface and more details.](https://github.com/davidpet/tutorials/commit/4e1e5c28cb813580e314770e8d62940b209d4147) - this could be done with `ng generate interface` but it's pretty trivial to just make a file and make an interface

[Step 1C: Add \<input\> with 2-way binding.](https://github.com/davidpet/tutorials/commit/4c0d6588286f09b0205eea8e30544da6c211486c)

[Step 1D: Adding test for two-way binding.](https://github.com/davidpet/tutorials/commit/f9c8bf27667b76e6d60d5d62f945fe831334ec69)


### 2. Display List


[Step 2A: Use mock heroes list.](https://github.com/davidpet/tutorials/commit/7644f8dd12b15b08195e90e78d84a68aab805435) - `*ngFor`

[Step 2B: Adding hero details with user edit.](https://github.com/davidpet/tutorials/commit/0e8514cfa3c2e932473adc29c29105b2eefc2727) - pipes, click event handler, `*ngIf`, `[class.selected]` syntax to dynamically set a css class based on TS code


### 3. Feature Component


[Step 3: Refactoring hero details to own component.](https://github.com/davidpet/tutorials/commit/06f3370fa74290eaca7e44a4189363f55887371f) - `@Input()` decorator


### 4. Add Services

`ng generate service hero` creates `HeroService` in the top-level subfolder along with the app (instead of in its own subfolder like a component). If you want it in a subfolder (eg. 'services'), you could just put that as part of the name (eg. `ng generate service services/hero`).

The generated service is configured to be provided in root (doesn't need module to know about it).


[Step 4A: Using service to get hero names.](https://github.com/davidpet/tutorials/commit/0b63abc96b8bed7fb8751aa4ed5cd31f23d8cb00) - creating and injecting service

[Step 4B: Refactoring hero service to use rxjs.](https://github.com/davidpet/tutorials/commit/bd288ebb7c0ed7c15b43addd8ce5d2cec2819c1e) - changing from synchronous return to observable to simulate async (more realistic), ngOnInit

[Step 4C: Adding message component and service.](https://github.com/davidpet/tutorials/commit/c2ba230d4cbb444739866f05be9331622f622ea9) - shared state between multiple components via a service

[Step 4D: Cleaning up subscription.](https://github.com/davidpet/tutorials/commit/ef4339bc3a5601c8a31856209ed7f6269b88fb2d) - not part of the tutorial - using the takeUntil pattern to clean up subscriptions, ngOnDestroy


### 5. Add Navigation


Routing is typically configured in its own module with `ng generate module app-routing --flat --module=app` which creates a module called `AppRoutingModule` and imports it into `AppModule`, without putting it into is own subfolder (--flat). You fill in the skeleton module contents as shown in the commits below.


[Step 5A: Moving heroes list to /heroes route.](https://github.com/davidpet/tutorials/commit/a81854fd85ae02aef6bb629075d427c9c91bd877) - adding routing (w/ _/heroes_ path) and moving component from main page into that route

[Step 5B: Adding link button to route to /heroes.](https://github.com/davidpet/tutorials/commit/8d87114da07a22143611af0d639cc27e1e5db558) - routing via button

- NOTE: `<nav>` tag is an HTML thing and not an Angular thing. See [here](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/nav)

[Step 5C: Adding dashboard view.](https://github.com/davidpet/tutorials/commit/67b86eaf75a4098a24778a6b2e0cea543d41483d) - new component+route, default route

[Step 5D: Moving hero details to parameterized url.](https://github.com/davidpet/tutorials/commit/d65ea04d6601dad20299c14f7987568c28fe3c3e) - adding route w/ parameter and component to read that parameter and show hero detail

[Step 5E: Adding back button in hero details.](https://github.com/davidpet/tutorials/commit/58583942ae6ddc99ffaeabe74c2c89a4b7a110a8) - navigation backwards


### 6. HTTP Data

Instead of going to a real server, we simulate an HTTP server locally with `angular-in-memory-web-api`, a package you can `npm install` to your project and use as shown in the commits. It has all the standard get, post, put, delete stuff out of the box and suburls with specific ids, etc. I believe that's called REST, but I've been vaguely corrected for using that term for HTTP stuff like this before, so I don't like to say it. The tutorial assumes all that stuff is already working and hard-coded to do what you'd expect (which it is in this case).


[Step 6A: Switching to HTTP for heroes service.](https://github.com/davidpet/tutorials/commit/0428b0a0fbf7e4921a3aafb937c679bd44a48270) - using InMemoryWebApi as HTTP server (w/ data) (HTTP GET), switching service to use HTTP (but still exposing same interface w/ observables)

[Step 6B: Adding http error handling.](https://github.com/davidpet/tutorials/commit/5c32cf18bdf6e5811b349cc56f43eb669288b9d5) - error handling in observables

[Step 6C: Use single ID call and fix logging.](https://github.com/davidpet/tutorials/commit/6bc0b513c46623781e35d60570037b2f7a89aca5) - HTTP GET on single ID

[Step 6D: Restore update ability (using http).](https://github.com/davidpet/tutorials/commit/cb5282f031c7ef5e1a0a854e1ddab6bc9bf969cc) - HTTP PUT, ReplaySubject (in the tests)

[Step 6E: Adding 'add' functionality.](https://github.com/davidpet/tutorials/commit/7a44b6b7effff5bccc31c783332e6d28883ee8b7) - HTTP POST

[Step 6F: Adding 'delete' and improving tests.](https://github.com/davidpet/tutorials/commit/bd860963f6438cbb8be9aaa774617dd598a61ea3) - HTTP DELETE, test readability

[Step 6G: Adding search bar component.](https://github.com/davidpet/tutorials/commit/e178b15228aa649e9272c102901a7d6966bb446d) - user input debouncing w/ rxjs, HTTP GET as a search

[Step 6H: Adding missing styles from final samples.](https://github.com/davidpet/tutorials/commit/318557eb2b0367e1201104b2de5d6bd2deea51f7) - some styles given along the way mismatch the final files given so I updated here

[Step 6I: Adding example getHeroNo404 and fixing a div.](https://github.com/davidpet/tutorials/commit/2b8f22e2d18852bef97adc78163efc7f2bd7397e) - fixing 2 more things that were different in the final files from what was shown along the way, one of which shows a different style of HTTP error handling regarding 404s


### 7. Final Cleanup


[Renaming project to show it's the tutorial + my extensions.](https://github.com/davidpet/tutorials/commit/23937e2619f74b1d75942f8ce42267c20ec61eb0) - shows an example of renaming an angular project including the internal references

WARNING: if VSCode is open while you do this, or when you pull it from another machine with an existing copy of the repo with these changes, all hell breaks loose. I don't know why yet.


## Template-Driven Form Tutorial

Similarly to above, I created commits following [this tutorial](https://angular.io/guide/forms). The GitHub code is [here](https://github.com/davidpet/tutorials/tree/master/Angular/template-driven-form).

I **didn't do tests** in this one because the stuff it dealt with is very simple from a testing perspective - there are no new concepts beyond what is in my tour of heroes tests. You would just query the DOM for the form control and keep sending text, checking classes, etc.

I'm also not going to re-explain things already explained above in Tour of Heroes.


### 1. Create Basic Form with Two-Way Binding


[1A: Default App (css + no routing)](https://github.com/davidpet/tutorials/commit/1b008c18e978610d6eac131341a61601f764fc3c)

[1B: Adding basic HTML form w/ mock data.](https://github.com/davidpet/tutorials/commit/83666ff7de8294756c715f94eb21019a93136970) - using [] to dynamically bind HTML attributes (not just inputs on components)

[1C: Two-way binding the form fields.](https://github.com/davidpet/tutorials/commit/c32202b33bea4d8840b1937cd363e9d38c97b642) - ngModel banana in a box again

[1D: Wrapping in form tag w/ ngForm ref.](https://github.com/davidpet/tutorials/commit/c7e3a72b90a707b79f955dc59002ae4c41752038) - ngForm directive as template reference variable

[1E: Random changes from the tutorial.](https://github.com/davidpet/tutorials/commit/638689e54a84c71a91390ca75c789a6dfdf94b66) - wrapping form groups (no real effect here - just something that may improve some forms)

[1F: Temporary proof of two-way binding.](https://github.com/davidpet/tutorials/commit/e5cb7dbf5a493d8929f13acbcdfeb85d4a4cb947) - how to see that your two-way binding is working, json pipe -> reverted in 1G (just the opposite of 1F)


### 2. Form Validation


[2A: Showing validation status color.](https://github.com/davidpet/tutorials/commit/db7359793c9f66ddcbfd9c82c47c021e982595d7) - styling based on form control state, global styling (but could have been in the actual global styling file instead of here)

[2B: Name is required again - what a ride!](https://github.com/davidpet/tutorials/commit/51158e61d3e3b248c2e1f363cfa1e4bc74465533) - marking a field as required for validation (HTML attribute, not angular thing - but used by ngForm)

[2C: Displaying error on name validation failure.](https://github.com/davidpet/tutorials/commit/fc991055c3768fa65e36b25860da00e1c9a42b43) - showing/hiding error based on validation state from ngForm template reference variable, also using 'hidden' instead of ngIf, template reference variable for ngModel


### 3. New Hero Button


[3A: New Hero button w/ val status reset.](https://github.com/davidpet/tutorials/commit/44e9743ef508e997684ef1527f0728de8006bdc2) - resetting form state


### 4. Form Submission


[4A: Submit button (does nothing yet).](https://github.com/davidpet/tutorials/commit/590953ce0b74101c4f2c1243475fcc637d7f45f3) - hiding button on invalid form, setting type to 'submit' for styling and so form knows it's the submit button

[4B: Marking submision state on submission.](https://github.com/davidpet/tutorials/commit/e9dc1f89f3361fecb2c5b53db7dbd1ec261b3ea2) - responding to form submissions in code

[4C: Hide the form in submitted state.](https://github.com/davidpet/tutorials/commit/4bdd967487633c8c643e148f79541b7df3878e49) - using hidden to hide the form on submission

[4D: Showing submitted hero and edit button.](https://github.com/davidpet/tutorials/commit/14f0448055d1109df94fd2889c48266360b719c4) - showing submitted content on submission, button to go back to form by changing the state


### 5. Final Tweaks


[5A: Making form clean on edit.](https://github.com/davidpet/tutorials/commit/ba6f929939e7f59f17882cefe39cc491105694f0) - using \*ngIf instead of \[hidden\] so that the form is pristine on edit

[5B: Imporiving layout by including container tag.](https://github.com/davidpet/tutorials/commit/6226f5e5dfb1cc2f415d7dede46e3aeab5e65ac9) - container div at top for styling purposes

[5C: Switching new and submit buttons to match tutorial.](https://github.com/davidpet/tutorials/commit/8647d3b4768e5c2b0f1c09b41a63e48e7d60f6ac) - swapping buttons to match final tutorial result

[Multiple validators.](https://github.com/davidpet/tutorials/commit/1c96f82566702cc02313de9f56b63854b43770ce) - multiple validators in template, showing conditional validation errors in DOM

[Adding custom validator.](https://github.com/davidpet/tutorials/commit/222a838cd9f2bc1ec46c86aab00656209ead768d) - custom validator, custom directive, showing custom error, regex use in validators


## Reactive Form Example

These commits are loosely based on [this tutorial](https://angular.io/guide/reactive-forms) and are located at [this repo](https://github.com/davidpet/tutorials/tree/master/Angular/reactive-form)

Not Included:

- **tests** - because same as template-driven (just DOM stuff)
  - you could access reactive form members via componentInstance but that goes against the spirit of testing the page
- **submitting** - because it's exactly the same as template-driven version

[Adding basic form elements.](https://github.com/davidpet/tutorials/commit/80b1cb3042e091d3fd75e8eea367c3ea47471307) - starts out the same as template-drive (adding html form controls with no bindings yet)

[Adding FormControl with two-way binding.](https://github.com/davidpet/tutorials/commit/f89505905b41b16f4025072b3c469ff111056411) - single FormControl instance (& template binding)

[Setting form control value from TS code.](https://github.com/davidpet/tutorials/commit/073bd629dedb4c03f07974bd24e524e9a85a16ce) - setting FormControl value from TS code

[Listening to observable of form control.](https://github.com/davidpet/tutorials/commit/b5f40cd630426ff667d2ee06a127c8b5454af885) - FormControl.valueChanges stream, async pipe

[Adding FormGroup w/ multiple levels of values.](https://github.com/davidpet/tutorials/commit/90cd84782a42667653c47fad33598a0e98219d4b) - FormGroup w/ multiple form controls, querying for children

[Nesting form group.](https://github.com/davidpet/tutorials/commit/6e13380129f8627e66c4294fa5275c854ad1740d) - FormGroup in FormGroup

[Using patchValue().](https://github.com/davidpet/tutorials/commit/7f3c6b01ce58a09cd000e3b296e6c52763949271) - patchValue() to only set subset of fields instead of whole form

[Protecting form keys from minification.](https://github.com/davidpet/tutorials/commit/3f599dc177ae6b00003a5907eea0fd272c37a5bf) - using `declare interface` and text keys to prevent name mangling of form control names, suppressing formatter from changing back

[Using FormBuilder.](https://github.com/davidpet/tutorials/commit/f746b56afaf53377d17fceef4685bfd57f71839c) - more terse FormGroup instantiation

[Adding validation.](https://github.com/davidpet/tutorials/commit/88fa8782e21dc36c61fbea7e96d1aa946de3ab1f) - validators in form control entries (as 2nd item in array), validation status value

[Dynamic form with FormArray.](https://github.com/davidpet/tutorials/commit/02da80a532c5ce9af7e2468fc9eb02e9b65461f2) - `*ngFor` with `index` iteration, FormArray, dynamically sized array of controls, controls automatically named by index, resetting array

[Non-nullable form values via FormControl() options.](https://github.com/davidpet/tutorials/commit/cb868af9ab17cc96f2ca91b1c01aa4c6ec3e0a55) - using options object of FormControl() instead of array syntax in builder, using non-nullable fields to get rid of |null boilerplate in interfaces

[Adding reset button and default value for 1 field.](https://github.com/davidpet/tutorials/commit/415e437402431c5aef580a1441a8b8ca38d9c5d6) - default form values on reset

[Using FormBuilder.nonNullable to cut down on boilerplate.](https://github.com/davidpet/tutorials/commit/8cfb47d6b14e24ff42955e79f27a385b7bde0196) - going back to array syntax by using the non-nullable calls

[These didn't need to be optional.](https://github.com/davidpet/tutorials/commit/4dbb03fdcac5eba980fcef46b64cbdafe5d13734) - taking out some unnecessary ? in form model interface

[Multiple validators and validation error text.](https://github.com/davidpet/tutorials/commit/cb7c3da7d9ee936bb720285a7cd265c622efe540) - multiple validators on 1 field by wrapping in an array, checking for specific errors in template

[Adding custom validator with regex.](https://github.com/davidpet/tutorials/commit/2d40eeb76acbbd1966f75b9d2c439cc0c4d3f18c) - defining and using custom form validator (easier than the template-driven equivalent)

[Using validation CSS classes.](https://github.com/davidpet/tutorials/commit/d9937084f6a3cc87c4e26fcc3956ca4b176c99d8) - adding 'required' class since not using html required attribute, using css classes to colorize like in template-driven tutorial


## Routing Tutorial

[These commits](https://github.com/davidpet/tutorials/tree/master/Angular/routing-tutorial) are based on the guide [here](https://angular.io/guide/routing-overview#angular-routing).

[Default app w/ routing enabled (and scss).](https://github.com/davidpet/tutorials/commit/d44834b9720e6b7fb28b989ac41699a68262be0c)

[Adding 2 routed components w/ links.](https://github.com/davidpet/tutorials/commit/2788afe4d7c42d5922a44dfde025d44b784681ea)

[Adding wildcard route with 404 component.](https://github.com/davidpet/tutorials/commit/0b6c4cbb475e65553a2c665c717bcde1ca061ba1)

[Adding default route.](https://github.com/davidpet/tutorials/commit/e20d4f8872985d902407a9a846016b8e7b7f2e19)

[Adding nested (child) routes.](https://github.com/davidpet/tutorials/commit/db5fbbf5ae1cc677651101d812b2db0a0ffe820e)

[Setting page titles for route paths.](https://github.com/davidpet/tutorials/commit/8ff7579529a6bce4e7ccc888b352de02960b8773)

[Adding promise for async child title.](https://github.com/davidpet/tutorials/commit/3e525f46f5b4c59e5d9ab6f39fc7399745b14c6d)

[Using title strategy to prepend title prefix.](https://github.com/davidpet/tutorials/commit/280b81ef5e37fee856162a2f83ca7efae51f1040)

[Adding title for the 404 page.](https://github.com/davidpet/tutorials/commit/c6efc4dc12941bfcc6964baa0c8107eeb70e90fd)

[Adding missing comment in title strategy.](https://github.com/davidpet/tutorials/commit/78c7feb50e16f07893702079edc07502042fba87)

[Adding relative route links.](https://github.com/davidpet/tutorials/commit/2c109d60382da9731c892fc6a930dbdc08ac82d6)

[Adding optional ID param and router.navigate.](https://github.com/davidpet/tutorials/commit/dc6adf0b4ed8d352c57d7ce87f274a212fe50b91)

[Adding default child route.](https://github.com/davidpet/tutorials/commit/f5dd677b25b024adc4ac809e70a707e47f75a2c0)

[Button style for links & highlight active route.](https://github.com/davidpet/tutorials/commit/f04410c8524a543f492c849a40cdf148a348a064)

[Adding info about router tracing.](https://github.com/davidpet/tutorials/commit/68567f1f420820d00eef03bc374b13e83af79352)

[Simulating "feature module" routing.](https://github.com/davidpet/tutorials/commit/10bb2d37808203e5b2d337d199531c0291165fb1)

[Adding matrix url params and router.navigate relative.](https://github.com/davidpet/tutorials/commit/cb846098715d3e3a96a8b7a8afd8812ce1fb5993)

[Adding 'data' fields and simulating animation config.](https://github.com/davidpet/tutorials/commit/cf9d8e2153379a71695d2fc3403152563dcdd554)

[Getting rid of occasional console error.](https://github.com/davidpet/tutorials/commit/e71f424fb614522f87dc3debce452d425f1fbfc5)

[Simplifying feature model inclusion.](https://github.com/davidpet/tutorials/commit/55b7ee2030e7a8d73a4ca9b82a974dd40a316120)

[Adding secondary outlet and routing.](https://github.com/davidpet/tutorials/commit/77c5009f1d696d6a9cece7d68a4719ae2430fb49) - summary includes description of how primary and secondary outlets function

[Adding secondary route clearing and clarifying some secondary details…](https://github.com/davidpet/tutorials/commit/fbf301607053fbd0a04273f479eca1a182b890a4)

[Adding activation guards (sync and async).](https://github.com/davidpet/tutorials/commit/3a716bad09e073ece4ba38205e909329cc89dc39)

Example of relative routing:

```
goToItems() {
  this.router.navigate(['items'], { relativeTo: this.route });
}
```

To use queryParams (no configuration needed):

```
this.route.snapshot.queryParams
```


## Angular Material Examples

[These commits](https://github.com/davidpet/tutorials/tree/master/Angular/angular-material) are based on a subset of the guides [here](https://material.angular.io/guides).

[Adding Angular Material (via ng add).](https://github.com/davidpet/tutorials/commit/1b1199572dd1d38dc018f5196c34fc92ad05d88f)

[Adding mat-slide-toggle component to app.](https://github.com/davidpet/tutorials/commit/9b69f2fcc512dea444c220b538f8b8b79df108f7)

[Adding mat-button.](https://github.com/davidpet/tutorials/commit/ec1e5f233598f7c3bb31a90035f2a136cadf7cf6)

[Adding address form from Angular Material schematic.](https://github.com/davidpet/tutorials/commit/b3c8ca557e2519f1ae30feba869b84aaac7c2b08)

[Adding mat nav bar from schematic.](https://github.com/davidpet/tutorials/commit/1c65c78b81b1924c9339ef40a8631cf6ae197a95)

[Adding dashboard from schematic.](https://github.com/davidpet/tutorials/commit/dd4d07871fb05ea34690e531f6dfb3941e6e1fb7)

[Adding table from schematic and changing button to raised.](https://github.com/davidpet/tutorials/commit/b6d29fa6cafd4257807f22aedbc3b78864dd2a18)

[Adding tree from schematic.](https://github.com/davidpet/tutorials/commit/fcc5ca76a6fc19235982a34772a0483028d6f2b7)

[Adding drag-and-drop from schematic.](https://github.com/davidpet/tutorials/commit/49b864db4c3dde84255b56c171d55696a4d67be2)


## Bazel Integration

### External Examples

[Aspect Build Angular Example](https://github.com/aspect-build/bazel-examples/tree/main/angular) - basic Angular/Bazel functionality via hybrid approach (the bulk of the exploration here)

[Aspect Build Angular Ngc Example](https://github.com/aspect-build/bazel-examples/tree/main/angular-ngc) - more low-level (and granular) Angular/Bazel Functionality (with sacrifice of schematics and other support from Angular team)

[Aspect Build Bazel Rules](https://github.com/aspect-build/rules_js) - rules used by the above examples that you can also use

[Angular DevKit Architect Docs](https://www.npmjs.com/package/@angular-devkit/architect-cli) - just a reference to help understand the rules

### My Tutorial Example

#### Location

https://github.com/davidpet/tutorials/tree/master/Angular/bazel-workspace

#### README

See the readme in the above repo for a final summary of the info about executing/building/testing in ng and bazel (hybrid).

#### Angular Part

[Creating empty workspace](https://github.com/davidpet/tutorials/commit/91609d48e506d43020f90f3110432726a217d46e#diff-4a5eb10b133b5fbb31ef9812481cfa3cff003ceed353cf11d80f93540b58536b) - demonstrates how you could create an empty workspace with minimal Angular files, such as the root of a _monorepo_

[Adding app to workspace](https://github.com/davidpet/tutorials/commit/4d9f7344f75a1109d35c955d56dba0059bfb7bef) - demonstrates adding a new app into a workspace (in this case selecting scss and no routing)

[Adding library to workspace](https://github.com/davidpet/tutorials/commit/7108df6962856e639f563bae940f728707920d70) - demonstrates adding a new library into a workspace

[Adding another library](https://github.com/davidpet/tutorials/commit/02871101128b47dcccf411c58025840cf476a5f4)

[Moving Projects into Subfolders](https://github.com/davidpet/tutorials/commit/a0b6e0bc3519d58461713f600183abcd68a035a0) - just the `git mv` part (forgot to check in angular.json)

[Moving Projects into Subfolders](https://github.com/davidpet/tutorials/commit/98387f0461a9b472152aede0835847f7745ede73) - the _angular.json_ part (left out of previous commit)

[Using Library in App](https://github.com/davidpet/tutorials/commit/b3e654d3baaff028039f8582e5252dd441e84c93) - importing component from library in module and component of app

[Using Common Library in App and Lib](https://github.com/davidpet/tutorials/commit/e3ec013bc75d3f953f7d7c85f874b54b8cbcc28a) - importing from common library and library and app at same time (works same way transparently)

[Adding Info about Production Builds](https://github.com/davidpet/tutorials/commit/10b1549e32e8031522aa7b79f422bd69a0f1f866)

[Adding readme note about npm dependencies](https://github.com/davidpet/tutorials/commit/c9318593c07973e818a9061c141ac5e7930bb8f2)

[Adding Aspect Examples to Readme](https://github.com/davidpet/tutorials/commit/98c410837a3f191307b7b9b8c65eb52253e409a0)

[Adding Info about Building Specific Projects](https://github.com/davidpet/tutorials/commit/0c45b1232e0364c736a1d58265291e04cc8e68f3)

#### Bazel Part - Getting Build & Serve to Work

[Adding Hybrid Bazel Support](https://github.com/davidpet/tutorials/commit/f2a54af6011825daa9f95331c3ae6b8664049b67) - the overall structure of adding bazel support (while maintaining angular CLI support) including a custom defs.bzl with a few macros, relying on Aspect Build libraries (at this commit, there is an error from jq still) - see comments in the files and README.md from the project for details.
NOTE: pnpm-lock.yaml has to be regenerated manually via `pnpm import`

[Removing leading Comment to Fix Jq Error](https://github.com/davidpet/tutorials/commit/4d03c068d068b8e296b92637ea0eae0833ce6957) - jq doesn't like comments in json files

[Fixing package.json path issue](https://github.com/davidpet/tutorials/commit/8b3a74e2f8f625827dc48eab08a007ff7cca6765) - because I put my app in _apps_ instead of in root like the Aspect example

[Fixing issue of app build artifacts not populating](https://github.com/davidpet/tutorials/commit/15c8845d5496272fdb1700ce7c534f1e1e04263b) - once again because of where I put my app

#### Bazel Part - Documentation and External Modules

[Info about Prod Builds](https://github.com/davidpet/tutorials/commit/8cb88d9464cb26a059ab014c69e54299da474c68)

[Info about Precommit Hooks](https://github.com/davidpet/tutorials/commit/db3528dd5a96b28c5b4678c84c52a8b3038a92e1)

[Demonstrating inclusion of npm module into app](https://github.com/davidpet/tutorials/commit/6bc1c59af978f7d6e38300aa31edb04600588919) - demonstrating installation of an npm dependency via npm and then pnpm import, then using it within the bazel build (already works for angular CLI build just by the npm install)

[Info about iBazel and Auto Refresh](https://github.com/davidpet/tutorials/commit/4540ba56b14bb68bd62896df5cdefc66da907758)

[Info about pnpm and Possible Precommit Hooks to Add](https://github.com/davidpet/tutorials/commit/cc8512425c6c93c4d05439a21e21c9297a62fcf1)

#### Bazel Part - Testing

[Getting unit tests to work](https://github.com/davidpet/tutorials/commit/4aeb20b598d62294eca3ff36729c85d02abeed92) - adding karma.conf.js (which our defs.bzl already assumed to begin with) and wiring them in via angular.json

[Fixing Some Paths in karma.conf.js](https://github.com/davidpet/tutorials/commit/2b77cc1e8d6623e1bd80684f611e3a9de82aceb3)

[Refactoring karma.conf.js files into common base](https://github.com/davidpet/tutorials/commit/9b651e84900513fe55da07e3004e37bb3d587773) - use of relative paths in karma.conf.js and adding config piece by piece

[Adding info about karma execution UI](https://github.com/davidpet/tutorials/commit/1954187081d40cab877766d37ad45d77528adeb1)

[README notes about Environments](https://github.com/davidpet/tutorials/commit/bcb7e84972800d4e26251f6e2094fd8c069da6b3)

#### Followup

[Example in Projects Repo](https://github.com/davidpet/projects/commit/1a59e019df541501bc2a5589a42786dbcb1119ff) - this commit shows an example of an app that's in a different folder than the name of the app in angular (which caused some finicky failures that I had to resolve here).


## Misc. Examples

These are from a [project](https://github.com/davidpet/tutorials/tree/master/Angular/misc-examples-for-snippets) I made just to put random snippets in as examples.


### Test State Leakage

[This test file](https://github.com/davidpet/tutorials/blob/master/Angular/misc-examples-for-snippets/src/app/test-state-leakage/test-state-leakage.component.spec.ts) demonstrates how state can unexpectedly leak between tests due to variables in `describe` blocks not being scoped the way they look.

- In summary, variables declared in `describe` blocks should be assumed to live through the whole test suite.
- but `beforeEach` in a `describe` block runs before every single test (from the parent `describe` down to the lowest
  - so you can declare the variables as `undefined` and fill them in within the `beforeEach` blocks
- If you really need to , you can use `afterEach` for cleanup.


### Component Inputs/Outputs

[These components](https://github.com/davidpet/tutorials/tree/master/Angular/misc-examples-for-snippets/src/app/inputs-outputs) demonstrate the basics of component input/output. In particular, none of the above tutorials dealt with @Output(), so I did that in here.

- In summary, inputs are bound by [] from the parent and can be seen as of `ngOnInit` in the lifecycle
  - It's not shown here, but the input can be changed dynamically (see change detection later).
- Inputs can be optional or required depending on how you deal with them.
- Outputs are subject-like streams that emit whatever type you want.
- The parent listens for individual values emitted with () syntax.
- The code that responds to an output uses $event to represent whatever was emitted.


### Change Detection

[These two classes](https://github.com/davidpet/tutorials/tree/master/Angular/misc-examples-for-snippets/src/app/change-detection) show a little bit of testing of change detection. The only really relevant thing to take from it for other things is how to set the **change detection strategy** on a component though.


### ViewChild()/ViewChildren()

[This component](https://github.com/davidpet/tutorials/tree/master/Angular/misc-examples-for-snippets/src/app/view-child) shows the basics of using @ViewChild() and @ViewChildren().

@ViewChild() can be a component type (to let you interact with the class) or a native element (and a few other options not demonstrated). It is changed on each change detection round to point to the first result found in the DOM of the component.

@Viewchildren() is a list version of @ViewChild(), and instead of being rebound on change detection, it emits an observable you can listen to.

Both are `undefined` in `ngOnInit` and then set for the first time in `afterViewInit`.


### Content Projection (transclusion)

[These components](https://github.com/davidpet/tutorials/tree/master/Angular/misc-examples-for-snippets/src/app/content-projection) show the very basics of content projection.

It shows single-slot (no selector), multi-slot (selector), calling a click handler from the projected code, the order and structure of projected content, etc.

For more complex scenarios, such as including multiple times or conditionally, see `ng-template`.


### Directives

[This folder](https://github.com/davidpet/tutorials/tree/master/Angular/misc-examples-for-snippets/src/app/directives) shows 2 directives: one using an attribute, and one using a class. They both work the same - only the selector is different.

A directive is basically a component without a template. It attaches input bindings and output events onto its host component.

The example above also show interacting with the DOM element the directive is attached to, within the change detection system of Angular.

Note that things like `@HostListener` and `ElementRef` could be used on components too if you wanted to.


### Component Lifecycle Hooks

You can see the list of hooks in order [here](https://angular.io/guide/lifecycle-hooks).

The most important ones to keep in mind are:

1. `ngOnInit` when the bindings are set.
2. `ngAfterContentInit` when content projection is done.
3. `ngAfterViewInit` when view children are bound.
4. `ngOnDestroy` when Angular is destroying the component (to let you clean up subscriptions, etc.).
5. `ngOnChanges` before `ngOnInit` and then over and over.


### json-server

Complete details are [here](https://angular.io/tutorial/first-app/first-app-lesson-14). You can `npm install` this package globally and run it on a json file to have it implement an HTTP server at a port on localhost that your app can talk to. That way you can mock the real HTTP service instead of having to use in-memory API.


### HTTP Authentication

See [here](https://angular.io/guide/http) for general information about how to modify HTTP headers in HTTPClient transparently.

To implement authentication, you would implement OAuth 2.0 or similar yourself (there are 3rd party npm packages that can help) and then add the token into the headers in the interceptor for http requests.


### failOnFirstTestError

By default, jasmine tests will continue after an `expect` fails so that you can see all the failures. If you don't want that to happen, you can set `failOnFirstTestError` in `angular.json`.


# SCSS

This is some concise syntax snippets I generated with ChatGPT.

```sass
// Variables
$primary-color: #007bff;
$font-size: 14px;

// Using external SCSS module with aliasing
@use 'path/to/external-module' as em;

// Nesting
.container {
  width: 100%;

  .title {
    font-size: $font-size;
    color: $primary-color;
  }

  .content {
    padding: 10px;
  }
}

// Mixins
@mixin center-element {
  display: flex;
  align-items: center;
  justify-content: center;
}

// Using external mixin from SCSS module
.element {
  @include em.centerElement;
}

// Classes and Selectors
.button {
  padding: 10px;
  font-size: $font-size;

  &:hover {
    background-color: lighten($primary-color, 10%);
  }

  &.disabled {
    opacity: 0.5;
    cursor: not-allowed;
  }

  .icon {
    margin-right: 5px;
  }
}

// Functions
@function calculate-area($width, $height) {
  @return $width * $height;
}

// Media Queries
@media (max-width: 768px) {
  .container {
    width: 80%;
  }
}

// Keyframes Animation
@keyframes fade-in {
  from {
    opacity: 0;
  }
  to {
    opacity: 1;
  }
}

// Using the Function
.area {
  width: 100px;
  height: 200px;
  area: calculate-area(100px, 200px);
}
```


# CSS

This is a concise (relatively) CSS snippet I generated iteratively with ChatGPT as a reference.

```css
/* Apply to all elements */
* {
  box-sizing: border-box;
}

/* Tag selector */
body {
  font-family: Arial, sans-serif;
  background-color: #f0f0f0;
  margin: 0;
  padding: 0;
}

/* ID selector */
#myButton {
  background-color: #008cba; /* Blue background */
  color: white; /* White text */
  border: none; /* No border */
  padding: 15px 32px; /* Padding */
  text-align: center; /* Centered text */
  text-decoration: none; /* No underline */
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer; /* Pointer cursor on hover */
}

/* Class selector */
.myClass {
  color: red;
}

/* Descendant selector */
nav ul {
  list-style-type: none; /* No bullets */
  padding: 0;
}

nav ul li {
  display: inline; /* Display inline */
  margin-right: 10px; /* Margin to the right */
}

/* Child selector */
nav > ul {
  border-bottom: 1px solid #ccc;
}

/* Adjacent sibling selector */
h1 + p {
  font-size: 1.2em;
}

/* General sibling selector */
h1 ~ p {
  color: green;
}

/* Attribute selector */
input[type="text"] {
  padding: 5px;
  border-radius: 4px; /* Rounded borders */
  border: 1px solid #ccc;
}

/* Pseudo-class selector */
a:hover {
  color: orange;
}

/* Pseudo-element selector */
p::first-letter {
  font-size: 2em;
  color: red;
}

/* Media query */
@media (max-width: 600px) {
  body {
    background-color: lightblue;
  }
}
```


# FlexBox

This is a concise Flexbox snippet generated with ChatGPT as a reference.

```css
/* The container becomes a flex container, laying out its children in a row */
.container {
  display: flex;
}

/* The children are laid out in a column instead of a row */
.container-column {
  display: flex;
  flex-direction: column;
}

/* The children can wrap onto multiple lines if they run out of space in the container */
.container-wrap {
  display: flex;
  flex-wrap: wrap;
}

/* Combines flex direction and wrap, laying out the children in a column and allowing them to wrap onto multiple lines */
.container-column-wrap {
  display: flex;
  flex-direction: column;
  flex-wrap: wrap;
}

/* Horizontally aligns the children at the center of the container */
.container-justify-center {
  display: flex;
  justify-content: center;
}

/* Horizontally distributes the children with equal space around them */
.container-justify-space-around {
  display: flex;
  justify-content: space-around;
}

/* Horizontally distributes the children with equal space between them */
.container-justify-space-between {
  display: flex;
  justify-content: space-between;
}

/* Vertically aligns the children at the center of the container */
.container-align-items-center {
  display: flex;
  align-items: center;
}

/* Vertically aligns the children at the start of the container */
.container-align-items-start {
  display: flex;
  align-items: flex-start;
}

/* Vertically aligns the children at the end of the container */
.container-align-items-end {
  display: flex;
  align-items: flex-end;
}

/* Vertically distributes multiple lines of children with equal space around them */
.container-align-content-space-around {
  display: flex;
  flex-wrap: wrap;
  align-content: space-around;
}

/* Allows a child to grow and take up remaining space in the container */
.item-flex-grow {
  flex-grow: 1;
}

/* Allows a child to shrink if necessary to prevent overflow of the container. "1" means the item can shrink proportionally to its flex-shrink factor. */
.item-flex-shrink {
  flex-shrink: 1;
}

/* Changes the order of a child. "1" means the item will appear after items with "0" or negative order, in source order. */
.item-order-1 {
  order: 1;
}
```


# CSS Grid

Grid often works better for __2D layouts__ than flexbox.  They can sometimes be used together (for instance for objects inside a master layout).

This snippet creates a simple grid with two rows and three columns, and places some content in various grid cells:

```html
<!DOCTYPE html>
<html>
<head>
    <title>Basic CSS Grid Layout</title>
    <style>
        .grid-container {
            display: grid;
            grid-template-columns: auto auto auto; /* Three columns */
            grid-template-rows: auto auto; /* Two rows */
            gap: 10px; /* Space between rows and columns */
            padding: 10px;
        }

        .grid-item {
            background-color: rgba(255, 255, 255, 0.8);
            border: 1px solid rgba(0, 0, 0, 0.8);
            padding: 20px;
            text-align: center;
        }
    </style>
</head>
<body>

<div class="grid-container">
    <div class="grid-item">1</div>
    <div class="grid-item">2</div>
    <div class="grid-item">3</div>
    <div class="grid-item">4</div>
    <div class="grid-item">5</div>
    <div class="grid-item">6</div>
</div>

</body>
</html>
```

- **.grid-container**: This class is applied to the container element. It defines the display as `grid`. `grid-template-columns` and `grid-template-rows` are used to define the layout of the grid. Here, it's set to have three columns and two rows.

- **.grid-item**: This class is for the items inside the grid. You can style these items as you like. In this example, they have a background color, a border, padding, and centered text.

- **gap**: This property sets the gap between grid rows and columns.

Each `<div>` with the class `grid-item` represents a cell in the grid, and they are automatically placed in the grid in the order they appear.

You can adjust the number of columns, rows, and the styling to suit your layout needs. CSS Grid is very powerful and can be used to create complex layouts with relative ease.

# HTML

This is a concise (relatively) HTML snippet I generated iteratively with ChatGPT as a reference.

```html
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Your Web App</title>
    <style>
      body {
        font-family: Arial, sans-serif;
      }
    </style>
  </head>
  <body>
    <!-- Header section with the main heading -->
    <header>
      <h1>Welcome to Your Web App</h1>
    </header>

    <!-- Navigation section -->
    <nav>
      <ul>
        <li><a href="#home">Home</a></li>
        <li><a href="#about">About</a></li>
      </ul>
    </nav>

    <!-- Main content section -->
    <main>
      <!-- Home section with an image and a button -->
      <section id="home">
        <h2>Home</h2>
        <div>
          <img
            src="path/to/your/image.jpg"
            alt="Description of image"
            width="500"
            height="600"
          />
          <button id="myButton">Click Me!</button>
        </div>
        <p>Welcome home, <span id="userName">Guest</span>!</p>
      </section>

      <!-- About section -->
      <section id="about">
        <h2>About</h2>
        <p>About us...</p>
      </section>

      <!-- Form section -->
      <form id="myForm">
        <label for="name">Name:</label><br />
        <input type="text" id="name" name="name" required /><br />
        <label for="email">Email:</label><br />
        <input type="email" id="email" name="email" required /><br />
        <label for="pwd">Password:</label><br />
        <input type="password" id="pwd" name="pwd" required /><br />
        <input type="submit" value="Submit" />
      </form>
    </main>

    <!-- Footer section -->
    <footer>&copy; 2023 Your Web App</footer>

    <!-- JavaScript section -->
    <script>
      // Prevent form from being submitted
      document
        .getElementById("myForm")
        .addEventListener("submit", function (event) {
          event.preventDefault();
          console.log("Form submission prevented.");
        });

      // Log a message when the button is clicked
      document
        .getElementById("myButton")
        .addEventListener("click", function (event) {
          console.log("Button clicked.");
        });
    </script>
  </body>
</html>
```


# HTML Forms

```html
<!DOCTYPE html>
<html>
<head>
    <title>Sample Form</title>
</head>
<body>
    <form action="/submit_form" method="post">
        <h2>Registration Form</h2>

        <!-- Text Input -->
        <label for="name">Name:</label>
        <input type="text" id="name" name="name" required><br><br>

        <!-- Email Input -->
        <label for="email">Email:</label>
        <input type="email" id="email" name="email" required><br><br>

        <!-- Password Input -->
        <label for="password">Password:</label>
        <input type="password" id="password" name="password" required><br><br>

        <!-- Number Input -->
        <label for="age">Age:</label>
        <input type="number" id="age" name="age" min="18" max="100"><br><br>

        <!-- Radio Buttons -->
        <label>Gender:</label>
        <input type="radio" id="male" name="gender" value="male">
        <label for="male">Male</label>
        <input type="radio" id="female" name="gender" value="female">
        <label for="female">Female</label><br><br>

        <!-- Checkbox -->
        <input type="checkbox" id="subscribe" name="subscribe" value="yes">
        <label for="subscribe">Subscribe to newsletter</label><br><br>

        <!-- Dropdown -->
        <label for="country">Country:</label>
        <select id="country" name="country">
            <option value="usa">United States</option>
            <option value="uk">United Kingdom</option>
            <option value="canada">Canada</option>
            <option value="australia">Australia</option>
        </select><br><br>

        <!-- Textarea -->
        <label for="bio">Bio:</label><br>
        <textarea id="bio" name="bio" rows="4" cols="50"></textarea><br><br>

        <!-- Submit Button -->
        <input type="submit" value="Register">

        <!-- Reset Button -->
        <input type="reset" value="Reset">
    </form>
</body>
</html>
```

- **Form Tag**: `<form action="/submit_form" method="post">` defines the form and specifies where to send the form data when submitted.

- **Input Fields**: Various types of input fields like text, email, password, number, radio buttons, and checkboxes.

- **Required Fields**: The `required` attribute in input fields to ensure they are filled out.

- **Labels**: The `<label>` tags are used for better accessibility and form readability.

- **Select (Dropdown)**: A dropdown list to select an option.

- **Textarea**: A larger area for text input, suitable for comments or messages.

- **Submit Button**: A button to submit the form.

- **Reset Button**: A button to clear all inputs in the form.

This form includes a mix of different controls that are commonly used in web forms, providing a good overview of the capabilities of HTML forms.